In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf


from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    LearningRateScheduler,
    ReduceLROnPlateau,
    EarlyStopping,
    TensorBoard,
)
from tensorflow.keras import backend as K
from tensorflow.keras.layers import GaussianNoise
from tensorflow.keras.regularizers import l2
from tensorflow.compat.v2.keras.layers import BatchNormalization
from tensorflow.python.client import device_lib
from utils import *
from models.resnet import *

In [ ]:
np.random.seed(1)
print(device_lib.list_local_devices())

# Loading images

In [ ]:
train_folder_img = "data/training/images/"
train_folder_gt = "data/training/groundtruth/"

filenames = os.listdir(train_folder_img)
n_train = len(filenames)
print(f"Load images and groundtruth for train: {n_train} ")

img_train = [load_img(train_folder_img+filenames[i]) for i in range(n_train)] 
gt_train = [load_img(train_folder_gt+filenames[i]) for i in range(n_train)] 

img_train = np.asarray(img_train)
gt_train = np.asarray(gt_train)

In [ ]:
val_folder_img = "data/validating/images/"
val_folder_gt = "data/validating/groundtruth/"

filenames = os.listdir(val_folder_img)
n_val = len(filenames)
print(f"Load images and groundtruth for validation: {n_val} ")

img_val = [load_img(val_folder_img+filenames[i]) for i in range(n_val)] 
gt_val = [load_img(val_folder_gt+filenames[i]) for i in range(n_val)] 

img_val = np.asarray(img_val)
gt_val = np.asarray(gt_val)

In [ ]:
img_train.shape

In [ ]:
gt_train.shape

In [ ]:
img_val.shape

In [ ]:
gt_val.shape

In [ ]:
X_train, Y_train = imag_rotation_aug(img_train, gt_train)

In [ ]:
X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)

In [ ]:
print(X_train.shape)
print(Y_train.shape)
n_train = Y_train.shape[0]

In [ ]:
X_val, Y_val = imag_rotation_aug(img_val,gt_val)

In [ ]:
X_val = np.asarray(X_val)
Y_val = np.asarray(Y_val)

In [ ]:
print(X_val.shape)
print(Y_val.shape)
n_val = Y_val.shape[0]

# ResNet-UNet Architecture

In [ ]:
data_aug_factor = 1
batch_normalization = False
activation = "relu"
dropout = 0
amsgrad = False
lr = 1e-4
BATCH_SIZE = 100
WINDOW_SIZE = 64
PATCH_SIZE = 16
EPOCHS = 80
STEPS_PER_EPOCH = 1
WIDTH = 448

In [ ]:
model = resnet_unet_model(
    shape=(64, 64, 3),
    batch_normalization=batch_normalization,
    activation=activation,
    dropout=dropout,
    amsgrad=amsgrad,
    lr=lr,
    BATCH_SIZE=BATCH_SIZE,
    WINDOW_SIZE=WINDOW_SIZE,
    PATCH_SIZE=PATCH_SIZE,
    EPOCHS=EPOCHS,
    STEPS_PER_EPOCH=STEPS_PER_EPOCH,
    WIDTH=WIDTH,
)

# Train the Model


In [ ]:
# Train the model
history = model.train(
    X_train, Y_train, n_train, X_val, Y_val, n_val
)
model.save("best_resnet_unet.h5")

In [ ]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure(figsize=(15, 10))
plt.plot(history.history["loss"][1:], label="train_loss")
plt.plot(history.history["val_loss"][1:], label="val_loss")

# If running on Google Colab use val_acc, if not val_accuracy
plt.plot(history.history["accuracy"][1:], label="train_acc")
plt.plot(history.history["val_accuracy"][1:], label="val_acc")
# plt.plot(history.history["acc"][1:], label="train_acc")
# plt.plot(history.history["val_acc"][1:], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("resnet_unet.pdf")
plt.show()

In [ ]:
# Instantiate the model
model = resnet_unet_model(
    shape=(64, 64, 3),
    batch_normalization=batch_normalization,
    activation=activation,
    dropout=dropout,
    amsgrad=amsgrad,
    lr=lr,
    BATCH_SIZE=BATCH_SIZE,
    WINDOW_SIZE=WINDOW_SIZE,
    PATCH_SIZE=PATCH_SIZE,
    EPOCHS=EPOCHS,
    STEPS_PER_EPOCH=STEPS_PER_EPOCH,
    WIDTH=WIDTH,
)

# Load the model
model.load("best_resnet_unet.h5")

model.model.summary()

# We add all test images to an array, used later for generating a submission
image_filenames = []
for i in range(1, 51):
    image_filename = "data/test_set_images/test_" + str(i) + "/test_" + str(i) + ".png"
    image_filenames.append(image_filename)

# Set-up submission filename
submission_filename = "resnet_unet.csv"

# Generates the submission
generate_submission(model, submission_filename, *image_filenames)